In [ ]:
#CF 相關係數的向量是text去算(而不是rate)

### Load the data

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
df_train=pd.read_csv("training_data.csv")
text_train=df_train['text']

In [4]:
star_train=df_train.stars

In [5]:
df_test=pd.read_csv("test_data.csv")
text_test=df_test.text

In [6]:
df_sub=pd.read_csv("sample_submission.csv",names=np.array(['id','stars']))
star_test=df_sub.stars

### Create the rating matrix

In [7]:
df_train.head()

,review_id,business_id,user_id,text,date,stars
0,3223,2055,2533,"Sometimes things happen, and when they do this...",2010-12-30,5
1,9938,4165,6371,I know Kerrie through my networking and we ben...,2011-04-26,5
2,7123,869,4929,Love their pizza!!!\nVery fresh. Their cannoli...,2012-09-28,5
3,3601,1603,2789,Being from NJ I am always on the prowl for my ...,2009-06-07,4
4,3948,2347,1245,We have tried this spot a few times and each v...,2011-02-20,4


In [33]:
df_train.user_id[df_train.user_id]

0       5
1       5
2       5
3       4
4       4
5       4
6       5
7       5
8       5
9       4
10      4
11      5
12      3
13      5
14      5
15      5
16      4
17      4
18      2
19      1
20      2
21      5
22      5
23      4
24      5
25      5
26      4
27      4
28      4
29      5
       ..
7967    4
7968    4
7969    4
7970    4
7971    4
7972    2
7973    4
7974    4
7975    1
7976    4
7977    5
7978    5
7979    4
7980    4
7981    5
7982    4
7983    5
7984    4
7985    5
7986    5
7987    2
7988    1
7989    5
7990    2
7991    5
7992    3
7993    3
7994    5
7995    4
7996    5
Name: stars, Length: 7997, dtype: int64

In [9]:
user_star_matrix=np.zeros((df_train.shape[0],1)) #rating from 1 to 5, and 0 means no rating
for i in df_train.itertuples():
    user_item_matrix[i[3]-1,i[2]-1]=i[6] #type is ndarray, index starts from 0, rows:user coln:item

In [30]:
print(np.sort(df_train.user_id.unique()))
print(np.sort(df_test.user_id.unique()).shape)

[   1    2    5 ... 6400 6401 6402]
(1706,)


### Text processing

#### tf-idf (ngram_range=(1, 1)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
tfidf=TfidfVectorizer(analyzer='word',ngram_range=(1, 1),min_df=0,stop_words='english')
tfidf.fit(text_train)
td_train=tfidf.transform(text_train)
td_test=tfidf.transform(text_test)

In [20]:
td_train=td_train.toarray()
td_test=td_test.toarray()

### Create the text similarity matrix

#### by cosine similarity measure

In [22]:
from sklearn.metrics.pairwise import linear_kernel

In [25]:
cos_sim=linear_kernel(td_test,td_train)

In [29]:
cos_sim.shape

(2003, 7997)

### Rating predict

#### Rtest,text=sum(cos_sim[test,train]* Rtrain,text)/sum(cos_sim[test,train]


In [57]:
star_pred_test=np.zeros((len(df_test)))

In [61]:
upper=0
lower=0

for i in range(len(df_test)):
    sim=cos_sim[i]
    sort_ind_sim=np.argsort(-sim)[:99]
    sort_sim=np.sort(sim)[:-100:-1]
    upper=sum(sort_sim*star_train[sort_ind_sim])
    lower=sum(sort_sim)

    if lower!=0:
        R=upper/lower
        if R<0:
            R=0
        if R>5:
            R=5
        star_pred_test[i]=(upper/lower)

In [63]:
import sklearn.metrics #inclusding the module of Rooted Mean Square Error(rmse)
import math

rmse=math.sqrt(sklearn.metrics.mean_squared_error(star_test, star_pred_test))

In [64]:
rmse

1.6689463036383547

In [71]:
np.mean(star_test-star_pred_test)

-0.8059634840914212